In [ ]:
from running_algo import *
import pandas as pd
from group_functions import *
from sub_super_script import *
from html_utils import *
from std_mean_ratio_method import *
from range_method import *
from formatting import *
from init_html import *
from get_unit_check import *
from plot_diurnal import *
from numbers_to_strings import *
import warnings
warnings.filterwarnings("ignore")
with open("regression.csv", "w", newline='') as fp:
    wr = csv.writer(fp, dialect='excel')
    wr.writerow(["rsq_ppb", "alpha_ppb", "beta_ppb","rsq", "alpha", "beta","pol", "station_name", "st_no"])
with open("mean_variation.csv", "w", newline='') as fp:
    wr = csv.writer(fp, dialect='excel')
    wr.writerow(["clean_mean", "raw_mean", "col", "station_name", "st_no"])
with open("percent_new_with_outlier.csv", "a", newline='') as fp:
    wr = csv.writer(fp, dialect='excel')
    wr.writerow(["clean_entries", "clean_entries_outliers", "available entries", "total entries", "station_name", "st_no", "pollutant"])   
    
appended_data_course = []
# see pd.concat documentation for more info

        
        
        
# # iterate over files in
# # that directory
directory = r'E:\MTech_Project\CPCB_Datavalidation\df_csv'
for filename in os.listdir(directory):
    path = os.path.join(directory, filename)

    # path =r"E:\MTech_Project\CPCB_Datavalidation\Data\Bandra, Mumbai - MPCB_2020.csv"
#     df = pd.read_csv(path)
#     df['dates']=pd.to_datetime(df['dates'], format="%Y-%m-%d %H:%M")
    
    
    true_df = get_formatted_df(path)[0]
    station_name = get_formatted_df(path)[1]
    year = '2021'






    
    df = true_df.copy(deep=True)
#     station_name = filename.replace("_2020.csv","")

    if true_df.empty:
        print('DataFrame is empty!', station_name, " ", year)
        continue


    filename=station_name+ '.html'
    start_html(station_name)
    print(station_name)

    PM25_LABEL = ('PM{}'.format(get_sub('2.5')))
    PM10_LABEL = ('PM{}'.format(get_sub('10')))
    NO2_LABEL = ('NO{}'.format(get_sub('2')))
    NOx_LABEL = ('NO{}'.format(get_sub('x')))
    SO2_LABEL = ('SO{}'.format(get_sub('2')))
    O3_LABEL = ('O{}'.format(get_sub('3')))
    st_no = numbers_to_strings(station_name)




    print(station_name)
    only_plots = true_df.copy(deep=True)

    local_df = true_df.copy(deep=True)
    local_df['date'] =  pd.to_datetime(local_df['dates']).dt.date
    local_df = local_df[local_df['date'].notna()]


#     import pdb; pdb.set_trace()
    try:
        local_df = group_plot(only_plots, local_df, 'PM25', PM25_LABEL,station_name,st_no)      
    except:
        print('error in PM25', station_name)
        pass
    
    try:local_df =group_plot(only_plots, local_df, 'PM10', PM10_LABEL,station_name,st_no)
    except:
        print('error in PM10', station_name)
        pass

    try:
        local_df =group_plot(only_plots, local_df, 'NOx', NOx_LABEL,station_name,st_no)
    except:
        print('error in NOx', station_name)
        pass
    
    try:
        local_df = group_plot(only_plots, local_df, 'NO', 'NO',station_name,st_no)
    except:
        print('error in NO', station_name)
        pass
    try:
        local_df = group_plot(only_plots, local_df, 'NO2', NO2_LABEL,station_name,st_no)
    except:
        print('error in NO2', station_name)
        pass
    try:
        local_df = group_plot(only_plots, local_df, 'SO2', SO2_LABEL,station_name,st_no)
    except:
        print('error in SO2', station_name)
        pass
    try:
        local_df =  group_plot(only_plots, local_df, 'Ozone', O3_LABEL,station_name,st_no)
    except:
        print('error in Ozone', station_name)
        pass

    try:
#         import pdb; pdb.set_trace()
        local_df['PM_course'] = local_df['PM10'] - local_df['PM25']

    
        if local_df[(local_df['PM_course'] < -5)].empty:print('No PM_course negative is empty!')
        else:
            course = local_df[(local_df['PM_course'] < -5)]
            course['station_name'] = station_name
            appended_data_course.append(course)
            
        local_df = local_df[(local_df['PM_course'] > -5)]
        
        
        local_df.to_csv(str("clean\\") + str(station_name) + ".csv")
        cleaning = get_ols_plot(local_df.NOx, local_df.NOx_clean, "NOx  cleaning", station_name)
        ppb = get_ols_plot(local_df.NOx_clean, local_df.NO2_clean + local_df.NO_clean, "reported nox, no+no2 reported", station_name)
        µg_m3 = get_ols_plot(local_df.NOx_clean, local_df['NO2_clean']*(1/1.88) + local_df['NO_clean']*(1/1.23)*1.9125, "reported nox, no+no2 in ppb assuming it in µg/m³",station_name)
        reg = [ppb[0], ppb[1], ppb[2], µg_m3[0], µg_m3[1], µg_m3[2], 'NOx', station_name, st_no]
        with open("regression.csv", "a", newline='') as fp:
            wr = csv.writer(fp, dialect='excel')
            wr.writerow(reg)
    except:
        
        print("error in NOx commpoenents")
#     import pdb; pdb.set_trace()
    
    
    print("Done with ", station_name)
    
    
appended_data_course = pd.concat(appended_data_course)    



# Unit finder

## Sum method

In [ ]:
directory = r'C:\Users\madhu\OneDrive - Indian Institute of Technology Bombay\Desktop\2021\clean'
import matplotlib.pyplot as plt
import numpy as np
import os
import csv
import pandas as pd
from numbers_to_strings import *
import warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
appended_data = []
for filename in os.listdir(directory):
    path = os.path.join(directory, filename)
    year = '2020'
    station_name = filename.replace(".csv", "")
    df = pd.read_csv(path)
    local_df = df.copy(deep =True)
    station_id = numbers_to_strings(station_name)
    print(station_id, station_name)
    if station_id == 46:
        continue

    

    df['ratio'] = (df['NO'] + df['NO2'])/df['NOx']
    try:
        temp = df[['NO2_clean_outliers', 'NOx_clean_outliers', 'NO_clean_outliers']].copy()
    except:
        print("error in ", station_name)
        continue
        pass
    
    def energy_class(x):

      if (x < 1.33) & (x > 1.25):
          return 'B'
      elif (x < 1.01) & (x > 0.98):
          return 'A'
      elif (x < 0.75) & (x > 0.66):
          return 'C'


      else:
          return np.nan
    

    df['score'] = df['ratio'].apply(energy_class)
    df['NO_unit'] = df['NO_clean_outliers']
    df['NO2_unit'] = df['NO2_clean_outliers']
    df['NOx_unit'] = df['NOx_clean_outliers']    
    
    df.loc[df['score'] == 'A', 'NO_unit'] = df['NO_unit']*1.23
    df.loc[df['score'] == 'A', 'NO2_unit'] = df['NO2_unit']*1.88
    df.loc[df['score'] == 'A', 'NOx_unit'] = (df['NO2_clean_outliers'] + df['NO_clean_outliers'])
    
    df.loc[df['score'] == 'C', 'NO_unit'] = df['NO_unit']*1.23
    df.loc[df['score'] == 'C', 'NO2_unit'] = df['NO2_unit']
    df.loc[df['score'] == 'C', 'NOx_unit'] = (df['NO2_clean_outliers']*(1/1.88) + df['NO_clean_outliers']) 
    
    local_df['score'] = df['score']

    groups = df.groupby('score')
    for name, group in groups:
        plt.plot(group['NOx'], group['NO2'] + group['NO'] , marker='o', linestyle='', markersize=1, label=name)
#     plt.show()
    plt.plot(local_df['NOx'], local_df['NO'] + local_df['NO2'] , marker='o', linestyle='', markersize=1, label=name)
#     plt.show()
    
    plt.legend()
    data = df[['NO', 'NO2', 'NOx','NO_unit','NO2_unit','NOx_unit','score']].copy()
    appended_data.append(data)

    plt.rcParams['figure.figsize'] = [5,5]
#     plt.show()
    lst = data.mean().to_list()
    lst.append(station_name)
    with open("new_no.csv", "a", newline='') as fp:
        wr = csv.writer(fp, dialect='excel')
        wr.writerow(lst)
#     print(station_name)
    
    

    
# see pd.concat documentation for more info
appended_data = pd.concat(appended_data)
# write DataFrame to an excel sheet 


    

    



## ML method

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from numbers_to_strings import *
import warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)


directory = r'C:\Users\madhu\OneDrive - Indian Institute of Technology Bombay\Desktop\2020\clean'
print("simple sum")
for filename in os.listdir(directory):
    path = os.path.join(directory, filename)
    year = '2020'
    station_name = filename.replace(".csv", "")
    df = pd.read_csv(path)
    local_df = df.copy(deep =True)
    station_id = numbers_to_strings(station_name)
    print(station_id, station_name)
    m_df = df
    try:
        
    
        m_df = m_df.dropna()     #
        X_data = m_df[['NO', 'NO2', 'NOx']].copy().iloc[:,:].values
        #correct NO2
        loaded_model = pickle.load(open('SVC_model_NO2.sav', 'rb'))        
        m_df['NO2_unit'] = loaded_model.predict(X_data)

        try:


            if ((m_df.groupby(['NO2_unit'])['NO2_unit'].count())['mi'] < (m_df.groupby(['NO2_unit'])['NO2_unit'].count())['ppb']*0.2):
                print("convert all NO2 to ppb")
                m_df["NO2_unit"] = 'ppb'
        except:
            pass
        try:

            if ((m_df.groupby(['NO2_unit'])['NO2_unit'].count())['ppb'] < (m_df.groupby(['NO2_unit'])['NO2_unit'].count())['mi']*0.2):
                print("convert all NO2 to mi")
                m_df["NO2_unit"] = 'mi'

        except:
            pass


        conditions = [
            (m_df["NO2_unit"] == 'ppb'),
            (m_df["NO2_unit"] == 'mi')]
        values = (1.88, 1)



        m_df["NO2_score"] = np.select(conditions, values)
        m_df["NO2_cpcb"] = m_df["NO2_score"] * m_df["NO2_clean_outliers"]

        #correct NO
        loaded_model = pickle.load(open('SVC_model_NO.sav', 'rb'))        
        m_df['NO_unit'] = loaded_model.predict(X_data)

        try:

            if ((m_df.groupby(['NO_unit'])['NO_unit'].count())['mi'] < (m_df.groupby(['NO_unit'])['NO_unit'].count())['ppb']*0.2):
                print("convert all NO to ppb")
                m_df["NO_unit"] = 'ppb'
        except:
            pass

        try:

            if ((m_df.groupby(['NO_unit'])['NO_unit'].count())['ppb'] < (m_df.groupby(['NO_unit'])['NO_unit'].count())['mi']*0.2):
                print("convert all NO to mi")
                m_df["NO_unit"] = 'mi'
        except:
            pass


        conditions = [
            (m_df["NO_unit"] == 'ppb'),
            (m_df["NO_unit"] == 'mi')]




        values = (1.23, 1)




        m_df["NO_score"] = np.select(conditions, values)

        m_df["NO_cpcb"] = m_df["NO_score"] * m_df["NO_clean_outliers"]

        m_df["NOx_cpcb"] = ((m_df["NO_cpcb"]/1.23) + (m_df["NO2_cpcb"]/1.88))*1.9125




        colors = {'ppb':'blue', 'mi':'orange'}
        plt.scatter(x=m_df['NO2'],y=m_df['NO2_cpcb'], c=m_df["NO2_unit"].map(colors))
        plt.show()
        path.replace("clean", "unit_clean")

    except:
        print("error in", station_name)
        pass



## Manual Method

In [ ]:
my_list_cal = []
my_list_repo = []
station_id = []
my_list_calno = []
my_list_calno2 = []
station_ID = []

act_no = []
act_no2 = []
act_nox = []

directory = r'E:\MTech_Project\CPCB_Datavalidation\aq_preprocessor\clean'
for filename in os.listdir(directory):
    path = os.path.join(directory, filename)
    year = '2020'
    station_name = filename.replace(".csv", "")
    df = pd.read_csv(path)
    local_df = df.copy(deep =True)
    try:
        df['dates']=pd.to_datetime(df['dates'], format="%Y-%m-%d %H:%M")
        df['NOx'] =  pd.to_numeric(df.NOx, errors='coerce')
        df['NO_clean'] =  pd.to_numeric(df.NO_clean, errors='coerce')  
        df['NO2_clean'] =  pd.to_numeric(df.NO2_clean, errors='coerce')  
        df['NOx_clean'] =  pd.to_numeric(df.NOx_clean, errors='coerce')



        df = df[df['NO_clean'].notna()]
        df = df[df['NO2_clean'].notna()]
        df = df[df['NOx_clean'].notna()]




        df['NOx_calculated'] = df['NO_clean'] + df['NO2_clean']
        df['NOx_reported'] = df['NOx_clean']

        st_no = numbers_to_strings(station_name)

        df['station_name'] = st_no
        
        station_ID.extend(df['station_name'].tolist())
        




        
        if (st_no in [1,2,3,4,5,6,8,11,23,24,30,39]) == True:
            df['group'] = 'A'
            df['NOx_calculated'] = df['NO_clean'] + df['NO2_clean']
            df['NOx_reported'] = df['NOx_clean']
        
            station_id.extend(df['group'].tolist())
         
        elif (st_no in [7,10, 13, 14, 16, 17, 20, 22,25,26,27,12, 33,34,17,36,37,38,20,22]) == True:
            df['group'] = 'B'
            df['NOx_calculated'] = (df["NO2_clean"]*1/1.88) + (df["NO_clean"]*1/1.23)
            df['NOx_reported'] = df['NOx_clean']
            
            
            station_id.extend(df['group'].tolist())
        elif (st_no in [35, 18,41]) == True:
            df['group'] = 'Mixed'
            station_id.extend(df['group'].tolist())             
        else:
            print(st_no)
            df['group'] = 'Unknown'
            station_id.extend(df['group'].tolist())
        
        my_list_repo.extend(df['NOx_reported'].tolist())
        my_list_cal.extend(df['NOx_calculated'].tolist())
        my_list_calno.extend(df['NO_clean'].tolist())
        my_list_calno2.extend(df['NO2_clean'].tolist())
        act_no.extend(df['NO'].tolist())
        act_no2.extend(df['NO2'].tolist())
        act_nox.extend(df['NOx'].tolist())



    
    except:
        pass
import seaborn as sns
import matplotlib.pyplot as plt

df1 = pd.DataFrame({'NO':act_no , 'NO2': act_no2, 'NOx':act_nox ,'NOx_reported': my_list_repo, 'NOx_calculated': my_list_cal,'station_id': station_id, 'station_ID': station_ID})
# df = df1[(df1['station_id'] == 'A') | (df1['station_id'] == 'B') ]
# sns.set(style="ticks")
# sns.set_style("ticks",{'axes.grid' : True})
# df['station_ID'] = df['station_ID'].astype(str)
# df = df.sort_values('station_ID')
# # df =df[(df['station_id'] == ('A' or 'B'))]

# df.rename(columns = {'station_id':'Condition'}, inplace = True)
# # Use palplot and pass in the variable:
# sns.set_palette("bright")
# sns_plot2 = sns.relplot(data=df, x='NOx_reported', y='NOx_calculated', hue='station_ID',col = 'Condition',s = 0.5, facet_kws={"legend_out": True})
# sns_plot2.set_axis_labels("Reported NOx (ppb)", "Calculated NOx (ppb)")
# sns_plot2.savefig('saving-a-seaborn-plot-as-pdf-file-300dpi.png', 
#            dpi=300)


df = df1[(df1['station_id'] == 'Unknown')]
sns.set(style="ticks")
sns.set_style("ticks",{'axes.grid' : True})
df['station_ID'] = df['station_ID'].astype(str)
df = df.sort_values('station_ID')
# df =df[(df['station_id'] == ('A' or 'B'))]

df.rename(columns = {'station_id':'Condition'}, inplace = True)
# Use palplot and pass in the variable:
sns.set_palette("bright")
sns_plot2 = sns.relplot(data=df, x='NOx_reported', y='NOx_calculated', hue='station_ID',col = 'Condition',s = 0.5, facet_kws={"legend_out": True})
sns_plot2.set_axis_labels("Reported "+ "$NO_{x}$" + "(unknown units)",  "$NO_{}$" + " (unknown units) +" + "$NO_{2}$" + " (unknown units)")


sns_plot2.savefig('Unknown-300dpi.png', 
           dpi=300)

df = df1[(df1['station_id'] == 'Mixed')]
sns.set(style="ticks")
sns.set_style("ticks",{'axes.grid' : True})
df['station_ID'] = df['station_ID'].astype(str)
df = df.sort_values('station_ID')
# df =df[(df['station_id'] == ('A' or 'B'))]

df.rename(columns = {'station_id':'Condition'}, inplace = True)
# Use palplot and pass in the variable:
sns.set_palette("bright")
sns_plot2 = sns.relplot(data=df, x='NOx_reported', y='NOx_calculated', hue='station_ID',col = 'Condition',s = 0.5, facet_kws={"legend_out": True})
# sns_plot2.set_axis_labels("Reported NOx (reported)", "NO (reported) + NO₂ (reported)")
sns_plot2.set_axis_labels("Reported "+ "$NO_{x}$" + "(reported)",  "$NO_{}$" + " (reported) +" + "$NO_{2}$" + " (reported)")

sns_plot2.savefig('Mixed-300dpi.png', 
           dpi=300)

df = df1[(df1['station_id'] == 'A')]
sns.set(style="ticks")
sns.set_style("ticks",{'axes.grid' : True})
df['station_ID'] = df['station_ID'].astype(str)
df = df.sort_values('station_ID')
# df =df[(df['station_id'] == ('A' or 'B'))]

df.rename(columns = {'station_id':'Condition'}, inplace = True)
# Use palplot and pass in the variable:
sns.set_palette("bright")
sns_plot2 = sns.relplot(data=df, x='NOx_reported', y='NOx_calculated', hue='station_ID',col = 'Condition',s = 0.5, facet_kws={"legend_out": True})
sns_plot2.set_axis_labels("Reported NOx (ppb → ppb)", "NO (ppb → ppb) + NO₂ (ppb → ppb)")
sns_plot2.set_axis_labels("Reported "+ "$NO_{x}$" + "(ppb → ppb)",  "$NO_{}$" + " (ppb → ppb) +" + "$NO_{2}$" + " (ppb → ppb)")

sns_plot2.savefig('A-300dpi.png', 
           dpi=300)

df = df1[(df1['station_id'] == 'B')]
sns.set(style="ticks")
sns.set_style("ticks",{'axes.grid' : True})
df['station_ID'] = df['station_ID'].astype(str)
df = df.sort_values('station_ID')
# df =df[(df['station_id'] == ('A' or 'B'))]

df.rename(columns = {'station_id':'Condition'}, inplace = True)
# Use palplot and pass in the variable:
sns.set_palette("bright")
sns_plot2 = sns.relplot(data=df, x='NOx_reported', y='NOx_calculated', hue='station_ID',col = 'Condition',s = 0.5, facet_kws={"legend_out": True})
sns_plot2.set_axis_labels("Reported "+ "$NO_{x}$" + "(ppb → ppb)",  "$NO_{}$" + " (µg/m³ → ppb) +" + "$NO_{2}$" + " (µg/m³ → ppb)")
sns_plot2.savefig('B-300dpi.png', 
           dpi=300)

# Simple sum for manual method

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from numbers_to_strings import *

directory = r'E:\MTech_Project\CPCB_Datavalidation\aq_preprocessor\clean'
print("simple sum")
for filename in os.listdir(directory):
    path = os.path.join(directory, filename)
    year = '2020'
    station_name = filename.replace(".csv", "")
    df = pd.read_csv(path)
    local_df = df.copy(deep =True)
    station_id = numbers_to_strings(station_name)
    try:
        df['dates']=pd.to_datetime(df['dates'], format="%Y-%m-%d %H:%M")
        df['NOx'] =  pd.to_numeric(df.NOx, errors='coerce')
        df['NO_clean'] =  pd.to_numeric(df.NO_clean, errors='coerce')  
        df['NO2_clean'] =  pd.to_numeric(df.NO2_clean, errors='coerce')  
        df['NOx_clean'] =  pd.to_numeric(df.NOx_clean, errors='coerce')
        df = df[df['NO_clean'].notna()]
        df = df[df['NO2_clean'].notna()]
        df = df[df['NOx_clean'].notna()]


        datasort = df
#         plt.figure(figsize=((4),(4)))
#         plt.grid(color = '#E5E4E2')

#     #         (df2['NO2']*(1/1.88) + df2['NO']*(1/1.23))*1.9125



#         pop = plt.scatter(datasort['NOx_clean']*(1/1.88),(datasort["NO2_clean"]*(1/1.88)  + datasort["NO_clean"]), color = 'purple', s = 0.2)
#         plt.ylabel('')
#         plt.legend(["Station ID: " + str(station_id)])

#         plt.show()
# Get the angles from 0 to 2 pie (360 degree) in narray object
        X = datasort['NOx_clean']
        X2 = datasort['NOx_clean']*1/1.88
        # Using built-in trigonometric function we can directly plot
        # the given cosine wave for the given angles
        Y1 = datasort["NO2_clean"] + datasort["NO_clean"]
        Y2 = (datasort["NO2_clean"]*1/1.88) + (datasort["NO_clean"]*1/1.23)
        Y3 =(datasort["NO2_clean"]*1/1.88) + (datasort["NO_clean"])


        # Initialise the subplot function using number of rows and columns
        figure, axis = plt.subplots(2, 2,figsize=(7, 7))

        # For Sine Function
        axis[0, 0].scatter(X, Y1, s =1)
        axis[0, 0].set_title("Case 1")
        axis[0, 0].grid(color = '#E5E4E2')  
        axis[0, 0].legend(["Station ID: " + str(station_id)])
        # For Cosine Function
        axis[0, 1].scatter(X, Y2, s=1)
        axis[0, 1].set_title("Case 2")
        axis[0, 1].grid(color = '#E5E4E2') 
        axis[0, 1].legend(["Station ID: " + str(station_id)])
        # For Tangent Function
        axis[1, 0].scatter(X, Y3, s=1)
        axis[1, 0].set_title("Case 3")
        axis[1, 0].grid(color = '#E5E4E2') 
        axis[1, 0].legend(["Station ID: " + str(station_id)])
        # For Tanh Function
        axis[1, 1].scatter(X2, Y3, s=1)
        axis[1, 1].set_title("Case 4")
        axis[1, 1].grid()
        axis[1, 1].legend(["Station ID: " + str(station_id)])
        # Combine all the operations and display
        plt.show()


    except:
        pass








